In [2]:
import numpy as np
from copy import deepcopy
from itertools import product

In [3]:
N_Rows = 6
N_Cols = 6
S_Positions = np.array([[(0,0), (0,1), (1,0), (1,1)],[(5,5), (5,4), (4,5), (4,4)]])
class HalmaState:
    def __init__(self, board_size, player_positions, turn, parent_state=None):
        self.board_size = board_size
        self.player_positions = np.array(player_positions)
        self.goal_positions = np.array([S_Positions[1], S_Positions[0]])
        self.turn = turn
        self.parent_state = parent_state
        self.child_states = []
        self.provis = None
        self.winner = -1   
        
    def __eq__(self, other):
        return np.array_equal(self.player_positions, other.player_positions)
                              
    def __hash__(self):
        return hash(str(player_positions))
    
    def check_terminal(self):
        comp = np.array([np.array_equal(self.goal_positions[0], self.player_positions[0]),
                       np.array_equal(self.goal_positions[1], self.player_positions[1])])
        if True in comp:
            self.winner = np.where(c == True)[0][0]+1
            return 1
        return 
    
    #def util(self):
    #    return self.winner
    
    def check_adjacent(self, idx):
        freefields = []
        occfields = []
        stack = np.vstack(self.player_positions)
        for x,y in product([-1,0,1], [-1,0,1]):
            newpos = np.add(idx,(x,y))  
            if not np.any(newpos < 0) and not np.any(newpos >= self.board_size):
                if not (newpos == stack).all(axis=1).any():                
                    freefields.append(tuple(newpos))
                elif tuple(newpos)!=tuple(idx):
                    occfields.append(tuple(newpos))
        return(freefields, occfields)
    
    
        
    def check_jump(self, pos, idx, visited = None):
        if visited is None:
            visited = set()
        stack = np.vstack(self.player_positions)
        val_pos = []
        _, occys = self.check_adjacent(pos)
        if occys:
            for p1,p2 in zip(product([-2,0,2], [-2,0,2]), product([-1,0,1], [-1,0,1])):
                newpos = np.add(pos, p1)
                if not np.any(newpos < 0) and not np.any(newpos >= self.board_size):
                    adoc = np.add(pos,p2)
                    if (adoc == occys).all(axis=1).any() and not (newpos == stack).all(axis=1).any() and tuple(newpos) not in visited:
                        visited.add(tuple(newpos))
                        ppos = deepcopy(self.player_positions[self.provis-1])
                        ppos[idx] = newpos
                        val_pos.append(np.array(ppos))
            n = [self.check_jump(k[idx], idx, visited) for k in val_pos]
            theres = [x for x in val_pos + n if len(x) > 0]
            theres = [item for sublist in theres for item in (sublist if isinstance(sublist, list) else [sublist])]
            return theres
        
    def moves_check(self, p = None):
        self.provis = deepcopy(self.turn)
        if p is not None:
            self.provis = p
        current_positions = deepcopy(self.player_positions[self.provis-1])
        possible = []
        for idx, i in  enumerate(current_positions):
            free_adjacents, occ_adjacents  = self.check_adjacent(i)
            for p in free_adjacents:
                ppos = deepcopy(current_positions)
                ppos[idx] = p
                possible.append(np.array(ppos))     
            jumps = self.check_jump(i, idx)
            if jumps:
                possible.extend(jumps)
        return possible
    
    def getpossiblestates(self, player = None):
        p = deepcopy(self.turn)
        if player is not None:
            p = player
        posis = []
        news = self.moves_check()
        for n in news:
            positions = deepcopy(self.player_positions)
            positions[p-1]=n
            posis.append(positions)
        return posis

    def make_children(self):
        newpos = self.moves_check()
        for i in newpos:
            positions = deepcopy(self.player_positions)
            positions[self.turn-1] = i
            t = 0
            if self.turn == 1:
                t = 2
            else:
                t = 1
            self.child_states.append(HalmaState(self.board_size, positions, t,self))
    
    def make_goal_board(self):
        board = np.zeros([self.board_size, self.board_size])
        for idx, player in enumerate(self.goal_positions):
            for pos in player:
                board[tuple(pos)] = idx+1
        return board
    
        
        
    def make_board(self, state = None, p = None):
        if state is None:
            board = np.zeros([self.board_size, self.board_size])
            for idx, player in enumerate(self.player_positions):
                for pos in player:
                    board[tuple(pos)] = idx+1
            return board
        else:
            board = np.zeros([self.board_size, self.board_size])
            positions = deepcopy(self.player_positions)
            positions[p-1]=state
            for idx, player in enumerate(positions):
                for pos in player:
                    board[tuple(pos)] = idx+1
            return board
    
    

In [4]:
def evaluation_function_1(game):
    board_goal = game.make_goal_board()
    board_state = game.make_board()
    #indexes_target_2 = np.where(board_goal == 2)
    indexes_target_2 = game.goal_positions[1]
    #indexes_in_2 = np.where(board_goal == 1)
    indexes_in_2 = game.goal_positions[0]
    #indexes_state_2 = np.where(board_state == 2)
    indexes_state_2 = game.player_positions[1]
    #indexes_state_1 = np.where(board_state == 1)
    indexes_state_1 = game.player_positions[0]
    index_list_inital_1 = []
    index_list_target_1 = []
    index_list_inital_2 = []
    index_list_target_2 = []
    index_list_2 = []
    index_list_1 = []
    for i in range(len(indexes_in_2[0])):
        index_in_2 = [indexes_in_2[0][i], indexes_in_2[1][i]]
        index_list_inital_2.append(index_in_2)
        index_tg_2 = [indexes_target_2[0][i], indexes_target_2[1][i]]
        index_list_target_2.append(index_tg_2)
        index_2 = [indexes_state_2[0][i], indexes_state_2[1][i]]
        index_list_2.append(index_2)
        index_1 = [indexes_state_1[0][i], indexes_state_1[1][i]]
        index_list_1.append(index_1)
        
    index_list_inital_1 = index_list_target_2
    index_list_target_1 = index_list_inital_2

    max_dist_1 = []
    max_dist_2 = []
    if index_list_inital_1 == index_list_target_1:
        game.winner = 1
        return 
    if index_list_inital_2 == index_list_target_2:
        game.winner = 2
        return
    
    for i in range(len(indexes_in_2[0])):
        dist_it_1 = []
        dist_it_2 = []
        
        if index_list_1[i] not in index_list_target_1:
            for j in range(len(indexes_in_2[0])):
                    dist_it_1.append(math.dist(index_list_1[i], index_list_target_1[j]))
                    
            max_dist_1.append(min(dist_it_1))
        else:
            max_dist_1.append(0)
            
        if index_list_2[i] not in index_list_target_2:
            for j in range(len(indexes_in_2[0])):
                    dist_it_2.append(math.dist(index_list_2[i], index_list_target_2[j]))
                
            max_dist_2.append(min(dist_it_2))
        else:
            max_dist_2.append(0)
        
    distances_1 = round(sum(max_dist_1), 5)
    distances_2 = round(sum(max_dist_2), 5)
    diff = distances_1 - distances_2
    return diff

In [17]:
def minimax(state, depth, maximizing, player, alpha, beta, evaluation_function):
    if depth == 0 or state.winner != -1:
        #print('hello')
        if evaluation_function == evaluation_function_3:
            return evaluation_function(state) * (-1 if player == 1 else 1)
        elif evaluation_function == evaluation_function_2:
            return evaluation_function(state)

    if maximizing:
        max_val = float('-inf')
        state.make_children()
        for move in state.child_states:
            evaluation = minimax(move, depth - 1, False, move.turn, alpha, beta, evaluation_function)
            max_val = max(max_val, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_val

    else:
        min_val = float('inf')
        #possible_moves = state.getpossiblestates(player)
        state.make_children()
        for move in state.child_states:
            #new_state = HalmaState(6, move, t)
            #winner = state.check_winner(new_state)
            #if winner != -1:
            #    return evaluation_function(new_state)
            evaluation = minimax(move, depth - 1, True, move.turn, alpha, beta, evaluation_function)
            min_val = min(min_val, evaluation)
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        return min_val

def execute_move(evaluate_func, depth):
    def execute_minimax_move_aux(game):
        best_move = None
        alpha = float('-inf')
        beta = float('inf')
        if evaluate_func == evaluation_function_3:
            best_eval = float('-inf')
            #possible_moves = state.getpossiblestates(player)
            #game.state.make_children()
            for move in game.state.getpossiblestates(game.state.turn):
                t = 0
                if game.state.turn == 1:
                    t = 2
                else:
                    t = 1
                new_state = HalmaState(6, move, t)
                # Change is_maximizing to true to see that pieces move outside of target area once there
                new_state_eval = minimax(new_state, depth - 1, False, game.state.turn, alpha, beta, evaluate_func)
                #print('new_state_eval: ',new_state_eval)
                
                if new_state_eval is not None:
                    if new_state_eval > best_eval:
                        best_move = new_state
                        best_eval = new_state_eval
                else:
                    best_move = new_state
                    #best_eval = float('inf')
            #print('best_eval: ', best_eval)
            game.state = best_move
        elif evaluate_func == evaluation_function_2:
            best_eval = float('inf')
            for move in game.state.getpossiblestates(game.state.turn):
                t = 0
                if game.state.turn == 1:
                    t = 2
                else:
                    t = 1
                new_state = HalmaState(6, move, t)
                # Change is_maximizing to true to see that pieces move outside of target area once there
                new_state_eval = minimax(new_state, depth - 1, True, game.state.turn, alpha, beta, evaluate_func)
                if new_state_eval:
                    if new_state_eval < best_eval:
                        best_move = new_state
                        best_eval = new_state_eval
                else:
                    best_move = new_state
            game.state = best_move
        #elif evaluate_func == evaluation_function_3
    return execute_minimax_move_aux

In [18]:
def evaluation_function_3(state):
    e_pos1 = S_Positions[1]
    e_pos2 = S_Positions[0]
    e_occ1 = ~(state.player_positions[0]== e_pos1).all(axis = 1)
    e_occ2 = ~(state.player_positions[1]== e_pos2).all(axis = 1)
    d1 = np.linalg.norm([min(np.linalg.norm(i-e_pos1, axis = 1)[e_occ1]) for i in state.player_positions[0]])
    #print(d1)
    d2 = np.linalg.norm([min(np.linalg.norm(i-e_pos2, axis = 1)[e_occ2]) for i in state.player_positions[1]])
    #print(d2)
    #if d1 == 0:
    #    state.check_terminal()
    return d1-d2

In [19]:
import time
import math

class ConnectHalmaGame:
    
    def __init__(self, player_1_ai, player_2_ai):
        #self.state = HalmaState(6, [[(0,0), (0,1), (3,3), (1,1)],[(5,5), (5,4), (1,4), (4,5)]], 1) # initial state
        self.player_1_ai = player_1_ai # store player 1 type (move selection method)
        self.player_2_ai = player_2_ai # store player 2 type (move selection method)
        
    def start(self, log_moves = False):
        self.state = HalmaState(6, S_Positions, 1)
        while True:
            # play the move
            # your code here
            #--------------------------------------------------#
            print('player', self.state.turn)
            if self.state.turn == 1:
                self.player_1_ai(self)
            else:
                self.player_2_ai(self)
            #--------------------------------------------------#
            
            if log_moves:
                print(game.state.make_board())
            
            # check the winner and end the game if so
            # your code here
            #--------------------------------------------------#
            if self.state.winner != -1:
                break
            #--------------------------------------------------#
        # print the winner of the game
        # your code here
        #--------------------------------------------------#
        print('winner', self.state.winner)
        if self.state.winner == 0:
            print("End of game! Draw!")
        else:
            print(f"End of game! Player {self.state.winner} wins!")
            
    def run_n_matches(self, n, max_time = 3600, log_moves = False):
        # utility function to automate n matches execution
        # should return the total distribution of players wins and draws
        # your code here
        #--------------------------------------------------#
        start_time = time.time()
        
        results = [0, 0, 0] # [draws, player 1 victories, player 2 victories]
        
        while n > 0 and time.time() - start_time < max_time:
            n -= 1
            #if n % 2 == 0:
            #    game.state.current_player = game.state.switch_player(game.state)
            game.start(log_moves)
            results[game.state.winner] += 1
            
        print("\n=== Elapsed time: %s seconds ===" % (int(time.time() - start_time)))
        print(f"  Player 1: {results[1]} victories")
        print(f"  Player 2: {results[2]} victories")
        print(f"  Draws: {results[0]} ")
        print("===============================")
        #--------------------------------------------------#

In [20]:
game = ConnectHalmaGame(execute_move(evaluation_function_3, 1), execute_move(evaluation_function_3, 1))
game.run_n_matches(1, 300, True)

player 1
[[0. 1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 2.]
 [0. 0. 0. 0. 2. 2.]]
player 2
[[0. 1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 2. 2.]
 [0. 0. 0. 0. 2. 0.]]
player 1
[[0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 1. 2. 2.]
 [0. 0. 0. 0. 2. 0.]]
player 2
[[0. 0. 0. 0. 0. 0.]
 [1. 1. 2. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 1. 2. 2.]
 [0. 0. 0. 0. 0. 0.]]
player 1
[[0. 0. 0. 0. 0. 0.]
 [0. 1. 2. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 1. 2. 2.]
 [0. 0. 0. 0. 0. 0.]]
player 2
[[0. 0. 0. 0. 0. 0.]
 [0. 1. 2. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 2. 2. 0.]
 [0. 0. 0. 1. 2. 0.]
 [0. 0. 0. 0. 0. 0.]]
player 1
[[0. 0. 0. 0. 0. 0.]
 [0. 1. 2. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 2. 0.]
 [0. 0. 0. 1. 2. 1.]
 [0. 0. 0. 0. 0. 0.]]
player 2
[[0. 0. 0. 0. 0. 0.]
 [0. 1. 2. 0. 0. 0

[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0

[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0

[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0

[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0

[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 2. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 2
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1.]]
player 1
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0

KeyboardInterrupt: 

In [1]:
[[[0,1], [1,0], [1,1], [2,2]], [[3,3], [4,4], [4,5], [5,4]]]

[[[0, 1], [1, 0], [1, 1], [2, 2]], [[3, 3], [4, 4], [4, 5], [5, 4]]]